In [1]:
import pandas as pd
from utils_analysis import prepare_scores, plot_correlation
import matplotlib.pyplot as plt
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
df = pd.read_csv("data/hotpotqa/test.csv")

# "results/bart/r1/bart-base_data_hotpotqa_checkpoint-26835_hotpotqa.json",
# "results/bart/r2/bart-base_data_hotpotqa_checkpoint-26835_hotpotqa.json",
# "results/bart/r3/bart-base_data_hotpotqa_checkpoint-26835_hotpotqa.json",
# "results/t5/r1/flan-t5-base_data_hotpotqa_checkpoint-26835_hotpotqa.json",
# "results/t5/r2/flan-t5-base_data_hotpotqa_checkpoint-26835_hotpotqa.json",
# "results/t5/r3/flan-t5-base_data_hotpotqa_checkpoint-26835_hotpotqa.json",

df_scores, beam_search_results = prepare_scores(
    "results/t5/r1/flan-t5-base_data_hotpotqa_checkpoint-26835_hotpotqa.json",
    df["target"],
    "f1",
)

/home/mila/f/floresl/beam-search/utils_analysis.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  abs(spearmanr(ground_truth_score, confidence_score_dict[str(k)]).statistic)  # type: ignore


In [3]:
df_scores.drop(["sentences", "dropout_sentences"], axis=1).corr(method="spearman")["f1"]

length_normalized_log_probs    0.301715
mean_token_entropy            -0.152426
dropout_bleu_variance          0.008336
dropout_meteor_score           0.290451
dropout_entropy               -0.342010
dropout_disagreement          -0.235966
f1                             1.000000
beam_score_ratios_90           0.429086
beam_score_log_probs_90       -0.233864
beam_score_top_k_90           -0.230181
beam_score_impt_wt_90          0.229041
Name: f1, dtype: float64

In [4]:
beam_score_log_probs = pd.DataFrame(beam_search_results["beam_score_log_probs"])
worst_rouge_idxs = df_scores.sort_values(by="f1").index[:100]
best_rouge_idxs = df_scores.sort_values(by="f1").index[::-1][:100]

In [5]:
# plt.bar(
#     x=list(range(1, 101)),
#     height=np.exp(beam_score_log_probs.loc[worst_rouge_idxs[0]]),
# )
# plt.xlabel("Beam Number")
# plt.ylabel("Joint Sequence Token Probability")

In [6]:
# plt.bar(
#     x=list(range(1, 101)),
#     height=np.exp(beam_score_log_probs.loc[best_rouge_idxs[-1]]),
# )
# plt.xlabel("Beam Number")
# plt.ylabel("Joint Sequence Token Probability")

In [7]:
from utils_tail_probs import softmax, tail_index
from scipy.stats import spearmanr
from scipy.spatial.distance import jensenshannon

In [8]:
tail_indices = []
for i in range(len(beam_score_log_probs)):
    probs = softmax(beam_score_log_probs.iloc[i].to_numpy(), temperature=0.05)
    tail_indices.append(tail_index(probs))

spearmanr(tail_indices, df_scores["f1"])

SignificanceResult(statistic=0.45084327032516935, pvalue=3.188911085633083e-51)

In [9]:
import json

baseline_probs = json.load(
    open(
        "/home/mila/f/floresl/beam-search/results/t5_baseline/google_flan-t5-base_hotpotqa.json",
        "r",
    )
)

In [10]:
baseline_log_probs = baseline_probs["beam_score_log_probs"]
baseline_log_probs = pd.DataFrame(baseline_log_probs).values

beam_score_log_probs = beam_score_log_probs.values

In [12]:
js_distances = []
for i in range(1000):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=0.05),
        softmax(baseline_log_probs[i], temperature=0.05),
    )
    js_distances.append(js)
spearmanr(js_distances, df_scores["f1"])

SignificanceResult(statistic=0.3658134678021087, pvalue=5.0446810980216386e-33)

In [13]:
js_distances = []
for i in range(1000):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=0.05), np.array([1 / 100] * 100)
    )
    js_distances.append(js)
spearmanr(js_distances, df_scores["f1"])

SignificanceResult(statistic=0.445056095679129, pvalue=8.220011082194255e-50)